In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from scipy.spatial.transform import Rotation

from typing import List
from omegaconf import OmegaConf
from hydra.utils import instantiate
from opr.pipelines.place_recognition import PlaceRecognitionPipeline
from opr.pipelines.place_recognition import TextLabelsPlaceRecognitionPipeline

In [3]:
def get_labels_by_id(labels: List[str], id: str):
    frame = labels[id]
    all_labels = [i["value"]["text"] for i in frame["back_cam_anno"] + frame["front_cam_anno"]]
    all_labels = sum(all_labels, [])
    return all_labels


def pose_to_matrix(pose):
    """From the 6D poses in the [tx ty tz qx qy qz qw] format to 4x4 pose matrices."""
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


def compute_error(estimated_pose, gt_pose):
    """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return dist_error, angle_error

In [4]:
from opr.datasets.itlp import ITLPCampus

QUERY_LABELS_PATH = "/home/docker_opr/Datasets/subsampled_data/indoor/00_2023-10-25-night/text_labels.json"
DB_LABELS_PATH = "/home/docker_opr/Datasets/subsampled_data/indoor/01_2023-11-09-twilight/text_labels.json"

QUERY_TRACK_DIR = "/home/docker_opr/Datasets/subsampled_data/indoor/00_2023-10-25-night"
DATABASE_TRACK_DIR = "/home/docker_opr/Datasets/subsampled_data/indoor/01_2023-11-09-twilight"

DEVICE = "cuda"

MODEL_CONFIG_PATH = "../configs/model/place_recognition/minkloc3d.yaml"
WEIGHTS_PATH = "../weights/place_recognition/minkloc3d_nclt.pth"

In [5]:
query_dataset = ITLPCampus(
    dataset_root=QUERY_TRACK_DIR,
    sensors=["lidar"],
    mink_quantization_size=0.5,
    load_semantics=False,
    load_text_descriptions=False,
    load_text_labels=False,
    load_aruco_labels=False,
    indoor=True,
)

db_dataset = ITLPCampus(
    dataset_root=DATABASE_TRACK_DIR,
    sensors=["lidar"],
    indoor=True,
)

with open(QUERY_LABELS_PATH, "rb") as f:
    query_labels = json.load(f)
    query_labels = json.loads(query_labels)


In [6]:
model_config = OmegaConf.load(MODEL_CONFIG_PATH)
model = instantiate(model_config)

pipe = TextLabelsPlaceRecognitionPipeline(
    db_labels_path=DB_LABELS_PATH,
    database_dir=DATABASE_TRACK_DIR,
    model=model,
    model_weights_path=WEIGHTS_PATH,
    device=DEVICE,
)

In [7]:
id = 0

timestamp = list(query_labels.keys())[id]
query_annos = get_labels_by_id(query_labels, timestamp)
print(f"query_annos = {query_annos}")

sample_data = query_dataset[id]
sample_pose_gt = sample_data.pop("pose") 

sample_output = pipe.infer(sample_data, query_annos)

print(f"sample_output.keys() = {sample_output.keys()}")
print(f"sample_output['idx'] = {sample_output['idx']}")
print(f"pose = {sample_output['pose']}")
print(f"pose_gt = {sample_pose_gt.numpy()}")

dist_error, angle_error = compute_error(sample_output["pose"], sample_pose_gt.numpy())
print(f"dist_error = {dist_error}, angle_error = {angle_error}")

query_annos = ['мфти', 'фровых технологий']
Using text labels
best_match_annos: ['центр цифровых технологий'], highest_similarity: 81
sample_output.keys() = dict_keys(['idx', 'pose', 'descriptor'])
sample_output['idx'] = 1
pose = [ 1.34200774  0.08154918 -0.08376353  0.02590458  0.01289916 -0.00886713
  0.99954186]
pose_gt = [ 0.95977867  0.11527952  0.17477702  0.00371521  0.02162522 -0.0095287
  0.99971384]
dist_error = 0.4626878207195888, angle_error = 2.7334507538083272
